In [183]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as BS
import sqlite3
import json
from tqdm import tqdm
import numpy as np
import re
import sqlalchemy
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
df = pd.read_csv('generalProfRatings.csv')
id = df.loc[0]['id']
id

2171

In [149]:
def getCourseInfo(profId, coursesDic,):
    url = 'https://www.ratemyprofessors.com/professor?tid=' + str(profId)
    req = requests.get(url, cookies = {'ccpa-notice-viewed-02':'yes', '_c_':'yes'})
    soup = BS(req.text, 'html.parser')
    try:
        prof = soup.select('#ratingsList')[0].find_all('li')
    except Exception as e:
        print(e)
        prof = []
    jj = prof
    try:
        #courseName = jj.find(attrs = {'class': 'RatingHeader__StyledClass-sc-1dlkqw1-2 gxDIt'}).text
        sampleSize = len(jj)
        comments = '\t\n'.join([(x.text) for x in jj for x in x.findAll(attrs = {'class': 'Comments__StyledComments-dzzyvm-0 gRjWel'}) ])
        qd = re.findall(r'\d+\.*\d*', jj[0].find(attrs = {'class':'RatingValues__StyledRatingValues-sc-6dc747-0 gFOUvY'}).text)
        quality = [round(float(re.findall(r'\d+\.*\d*', x.text)[0]), 2) for x in jj for x in x.findAll(attrs = {'class': 'RatingValues__RatingContainer-sc-6dc747-1 DObVa'})[:1]]
        qualityStd = round(np.std(quality), 3)
        difficulty = [round(float(re.findall(r'\d+\.*\d*', x.text)[0]), 2) for x in jj for x in x.findAll(attrs = {'class': 'RatingValues__RatingContainer-sc-6dc747-1 DObVa'})[1:]]
        difficultyStd = round(np.std(difficulty), 3)
        commentsRatings = {'comment': comments, 'sampleSize': sampleSize, 'qualityAvg': round(np.mean(quality), 3), 'difficultyAvg': round(np.mean(difficulty), 3), 'qualityStd':qualityStd, 'difficultyStd':difficultyStd }
        try:
            coursesDic[profId] = commentsRatings
        except Exception as e:
            print(e)
            coursesDic[profId] = commentsRatings
    except Exception as e:
        #print(e)
        pass

In [152]:
profDic = {}
for _, ii in enumerate(df.id):
    print('[%d]\t\t[%d]' % (_, ii))
    getCourseInfo(ii, profDic)


[0]		[2171]
[1]		[18708]
[2]		[18711]
[3]		[37951]
[4]		[41264]
[5]		[66753]
[6]		[70976]
[7]		[84859]
[8]		[85026]
[9]		[85153]
[10]		[88367]
[11]		[98707]
[12]		[105411]
[13]		[107876]
[14]		[109027]
[15]		[110014]
[16]		[111255]
[17]		[111269]
[18]		[111274]
list index out of range
[19]		[129601]
[20]		[134376]
[21]		[136143]
[22]		[139906]
[23]		[143149]
[24]		[143155]
[25]		[146536]
[26]		[153827]
[27]		[155314]
[28]		[157346]
[29]		[159296]
[30]		[159368]
[31]		[160809]
[32]		[164719]
[33]		[165885]
[34]		[177733]
[35]		[184255]
[36]		[187881]
[37]		[190132]
[38]		[195465]
[39]		[197711]
[40]		[197988]
[41]		[198918]
[42]		[214243]
[43]		[218972]
[44]		[221926]
[45]		[226126]
[46]		[229005]
[47]		[229010]
[48]		[197997]
[49]		[246012]
[50]		[246750]
[51]		[246762]
[52]		[247988]
[53]		[250328]
[54]		[250333]
[55]		[252129]
[56]		[254008]
[57]		[262333]
[58]		[264134]
[59]		[266382]
[60]		[266476]
[61]		[268538]
[62]		[272742]
[63]		[272743]
[64]		[274934]
[65]		[278801]
[66]		[27

In [182]:
prof = pd.DataFrame(profDic).T
prof['profID'] = prof.index
prof.insert(0, 'profID', prof.pop('profID'))
prof = prof.reset_index(drop = True)
prof

,profID,comment,sampleSize,qualityAvg,difficultyAvg,qualityStd,difficultyStd
0,2171,This class was mandatory. Prof Keefer is a nic...,8,4.5,2.833,1.118,0.687
1,18708,He was a great professor who had a tough subje...,8,2.833,4.5,1.28,0.764
2,18711,There is so much reading... The feedback on ou...,13,3.45,3.4,1.404,1.356
3,37951,I took this elective for fun $ am so happy I d...,9,4.5,2.0,0.463,0.756
4,41264,"LIFESCIENCE CORE REQUIREMENT: 22% EXAM 1, 22% ...",26,3.2,3.2,1.528,1.208
...,...,...,...,...,...,...,...
2851,1052652,Imagine reading an article from The New York T...,26,3.6,3.85,1.908,1.276
2852,1388150,Geroulanos expects a lot from his students but...,26,3.85,4.0,1.352,0.548
2853,2649675,One of my favorite classes at NYU. Professor B...,26,5.0,2.7,0.0,0.714
2854,2118628,shitttt\n\t\n10/10 recommend not taking this c...,8,1.333,4.5,0.745,0.5


In [184]:
conn_string = 'mysql://{user}:{password}@{host}:{port}/{db}?charset={encoding}'.format(
    user='daily_screamer', 
    password='sF5pxbMLBg8=', 
    host = 'jsedocc7.scrc.nyu.edu', 
    port     = 3306, 
    encoding = 'utf8',
    db = 'daily_screamer'
)
engine = sqlalchemy.create_engine(conn_string)

In [188]:
df = df.iloc[:, 1:]

In [190]:
prof.to_sql('RMPcommentsAndRatings', con = engine, index =False)

In [191]:
df.to_sql('RMPgeneralProfInfo', con = engine, index = False)